In [2]:
import pandas as pd
results = pd.read_csv('vit_attack_results_2.csv')
import matplotlib.pyplot as plt
import numpy as np

In [3]:
no_pos = results[results["pos_emb_ratio"] == 0.0]
one_eighth = results[results["pos_emb_ratio"] == 0.125]
one_fourth = results[results["pos_emb_ratio"] == 0.25]
one_half = results[results["pos_emb_ratio"] == 0.5]
one_fifth = results[results["pos_emb_ratio"] == 0.2]

In [4]:
results.columns

Index(['Patch_Size', 'Dim', 'Depth', 'Heads', 'pos_emb_ratio', 'Accuracy_old',
       'Accuracy_new', 'phase_strong_old', 'magnitude_strong_old',
       'normal_strong_old', 'FGSM_strong_old', 'fourier_strong_old',
       'phase_strong_new', 'magnitude_strong_new', 'normal_strong_new',
       'FGSM_strong_new', 'fourier_strong_new'],
      dtype='object')

In [5]:
all_attacks = ["phase_strong", "magnitude_strong", "normal_strong", "FGSM_strong", "fourier_strong"]

In [6]:
for attack in all_attacks:
    best_ratio = 0.0
    best_increase = 0.0
    for ratio in [0.0, 0.125, 0.2, 0.25, 0.5]:
        attack_old = f"{attack}_old"
        attack_new = f"{attack}_new"
        old = results[results["pos_emb_ratio"] == ratio][attack_old].values
        new = results[results["pos_emb_ratio"] == ratio][attack_new].values
        inc = new-old
        mean_inc = np.mean(inc)
        std_inc = np.std(inc)
        worst_inc = np.min(inc)
        best_inc = np.max(inc)
        if mean_inc > best_increase:
            best_increase = mean_inc
            best_ratio = ratio
            best_std = std_inc
            best_best_inc = best_inc
            best_worst_inc = worst_inc
    print(f"Best ratio for {attack}: {best_ratio:.3f}, increase: {best_increase:.3f} std: {best_std:.3f}, best: {best_best_inc:.3f}, worst: {best_worst_inc:.3f}")



Best ratio for phase_strong: 0.125, increase: 0.953 std: 1.643, best: 5.560, worst: -3.720
Best ratio for magnitude_strong: 0.125, increase: 0.995 std: 1.581, best: 4.930, worst: -3.310
Best ratio for normal_strong: 0.125, increase: 1.451 std: 2.248, best: 8.780, worst: -4.950
Best ratio for FGSM_strong: 0.500, increase: 0.194 std: 0.924, best: 2.160, worst: -2.570
Best ratio for fourier_strong: 0.500, increase: 0.239 std: 0.656, best: 2.320, worst: -1.410


In [7]:
# results_table = pd.DataFrame(columns=["ratio", "attack", "old", "new", "inc"])
results_table = pd.DataFrame(columns=["ratio", "attack", "inc"])
for ratio in [0.0, 0.125, 0.2, 0.25, 0.5]:
    for attack in all_attacks:
        attack_old = f"{attack}_old"
        attack_new = f"{attack}_new"
        old = results[results["pos_emb_ratio"] == ratio][attack_old].values
        new = results[results["pos_emb_ratio"] == ratio][attack_new].values
        inc = new-old
        old_mean = np.mean(old)
        new_mean = np.mean(new)
        inc_mean = np.mean(inc)
        row = {
            "ratio": ratio,
            "attack": attack,
            # "old": old_mean,
            # "new": new_mean,
            "inc": inc_mean
        }
        results_table = pd.concat([results_table, pd.DataFrame([row])], ignore_index=True)

results_table = results_table.pivot(index="ratio", columns="attack", values="inc")

/tmp/ipykernel_276380/312580925.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_table = pd.concat([results_table, pd.DataFrame([row])], ignore_index=True)


In [8]:
results_table.to_csv("vit_attack_results_table.csv")

In [9]:
# Finding best model
new_accs = results["Accuracy_new"].values
print("New model accuracy: ", new_accs.max())
best_model = results[results["Accuracy_new"] == new_accs.max()]
display(best_model)

New model accuracy:  72.15


,Patch_Size,Dim,Depth,Heads,pos_emb_ratio,Accuracy_old,Accuracy_new,phase_strong_old,magnitude_strong_old,normal_strong_old,FGSM_strong_old,fourier_strong_old,phase_strong_new,magnitude_strong_new,normal_strong_new,FGSM_strong_new,fourier_strong_new
2,4.0,256.0,8.0,8.0,0.2,70.49,72.15,18.4,18.51,25.55,13.46,0.37,19.66,19.97,27.55,13.81,0.19


In [17]:
for ratio in [0.0, 0.125, 0.2, 0.25, 0.5]:
    acc_old = results[results["pos_emb_ratio"] == ratio]["Accuracy_old"].values
    acc_new = results[results["pos_emb_ratio"] == ratio]["Accuracy_new"].values
    inc = acc_new - acc_old
    mean_inc = np.mean(inc)
    std_inc = np.std(inc)
    print(f"Ratio: {ratio:.3f}, inc: {mean_inc:.3f} std: {std_inc:.3f}")

Ratio: 0.000, inc: -2.006 std: 1.801
Ratio: 0.125, inc: 0.872 std: 1.164
Ratio: 0.200, inc: 0.866 std: 1.028
Ratio: 0.250, inc: 0.559 std: 0.998
Ratio: 0.500, inc: -0.477 std: 1.301


In [11]:
results["fourier_strong_old"].values.mean()

np.float64(1.7775999999999998)

In [12]:
np.mean(np.array(no_pos["Accuracy_new"] - no_pos["Accuracy_old"]))

np.float64(-2.0056)

In [13]:
np.mean(np.array(one_eighth["Accuracy_new"] - one_eighth["Accuracy_old"]))

np.float64(0.8721000000000001)

In [14]:
np.mean(np.array(one_fourth["Accuracy_new"] - one_fourth["Accuracy_old"]))


np.float64(0.5594000000000005)

In [15]:
np.mean(np.array(one_half["Accuracy_new"] - one_half["Accuracy_old"]))

np.float64(-0.4767999999999997)

In [16]:
np.mean(np.array(one_fifth["Accuracy_new"] - one_fifth["Accuracy_old"]))

np.float64(0.8656000000000006)